In [9]:
%matplotlib inline
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import ParameterGrid
import datetime
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [6]:
df_train=pd.read_csv('fraud_train.csv', parse_dates=['date'])

In [7]:
df_train_clean=df_train.loc[df_train['label']!=-1]
del df_train

In [10]:
y=df_train_clean['label']
X=df_train_clean.drop(['id','label','date'],axis=1)
cut_date_train=datetime.datetime(2017,10,20)
cut_date_test=datetime.datetime(2017,11,1)

In [11]:
x_train=X[df_train_clean.date<=cut_date_train]
x_test=X[df_train_clean.date>=cut_date_test]
y_train=y[df_train_clean.date<=cut_date_train]
y_test=y[df_train_clean.date>=cut_date_test]

In [12]:
num_values=1000
category_feature=[]
for i in range(X.shape[1]):
    num = X['f'+str(i+1)].unique().shape[0]
    if num <=num_values:
        category_feature.append(i)

In [13]:
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [14]:
feature_name=['feasture_'+str(col+1) for col in range(x_train.shape[1])]
lgb_train=lgb.Dataset(x_train,y_train,free_raw_data=False,feature_name=feature_name, categorical_feature=category_feature)
lgb_valid=lgb.Dataset(x_test,y_test,reference=lgb_train,free_raw_data=False,feature_name=feature_name, categorical_feature=category_feature)


In [39]:
params_grid = {
    'boosting_type': ['gbdt'],
    'objective': ['binary'],
    'metric': ['auc'],
    'train_metric': [True],
    'num_leaves': np.arange(30,60,5),
    'early_stopping_round': [5,10],
    'learning_rate': [0.1,0.05,0.01],
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    'device': ['cpu'],
    'verbose': [0]
}
params_list=ParameterGrid(params_grid)

In [40]:
best_params=[]
best_score=0
for params in list(params_list):
    gbm=lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=[lgb_train, lgb_valid], valid_names=['train_set', 'valid_set'], feval=atec_metric)
    y_pred=gbm.predict(x_test)
    score=atec_metric(y_pred,lgb_valid)[1]
    if score > best_score:
        best_score=score
        best_params=params

C:\Users\Dong\Anaconda3\lib\site-packages\lightgbm\engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Dong\Anaconda3\lib\site-packages\lightgbm\basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	train_set's auc: 0.934009	train_set's atec_metric: 0.387266	valid_set's auc: 0.8978	valid_set's atec_metric: 0.288372
Training until validation scores don't improve for 5 rounds.
[2]	train_set's auc: 0.942338	train_set's atec_metric: 0.425732	valid_set's auc: 0.909921	valid_set's atec_metric: 0.362791
[3]	train_set's auc: 0.944348	train_set's atec_metric: 0.457906	valid_set's auc: 0.913123	valid_set's atec_metric: 0.388857
[4]	train_set's auc: 0.944877	train_set's atec_metric: 0.458951	valid_set's auc: 0.913621	valid_set's atec_metric: 0.395736
[5]	train_set's auc: 0.958378	train_set's atec_metric: 0.46778	valid_set's auc: 0.948813	valid_set's atec_metric: 0.384787
[6]	train_set's auc: 0.959237	train_set's atec_metric: 0.474934	valid_set's auc: 0.947461	valid_set's atec_metric: 0.415019
[7]	train_set's auc: 0.960003	train_set's atec_metric: 0.480928	valid_set's auc: 0.949176	valid_set's atec_metric: 0.412016
[8]	train_set's auc: 0.960211	train_set's atec_metric: 0.482386	valid_set'

[28]	train_set's auc: 0.978596	train_set's atec_metric: 0.598978	valid_set's auc: 0.96796	valid_set's atec_metric: 0.483818
[29]	train_set's auc: 0.978899	train_set's atec_metric: 0.60279	valid_set's auc: 0.968351	valid_set's atec_metric: 0.480911
[30]	train_set's auc: 0.979109	train_set's atec_metric: 0.605799	valid_set's auc: 0.968027	valid_set's atec_metric: 0.480911
[31]	train_set's auc: 0.97955	train_set's atec_metric: 0.608531	valid_set's auc: 0.968314	valid_set's atec_metric: 0.483236
[32]	train_set's auc: 0.979717	train_set's atec_metric: 0.610828	valid_set's auc: 0.968047	valid_set's atec_metric: 0.486628
[33]	train_set's auc: 0.980282	train_set's atec_metric: 0.614468	valid_set's auc: 0.96889	valid_set's atec_metric: 0.486822
[34]	train_set's auc: 0.98137	train_set's atec_metric: 0.617648	valid_set's auc: 0.970582	valid_set's atec_metric: 0.486337
[35]	train_set's auc: 0.982154	train_set's atec_metric: 0.621885	valid_set's auc: 0.970678	valid_set's atec_metric: 0.48905
[36]	t

[13]	train_set's auc: 0.966514	train_set's atec_metric: 0.548375	valid_set's auc: 0.958289	valid_set's atec_metric: 0.428101
[14]	train_set's auc: 0.96665	train_set's atec_metric: 0.554955	valid_set's auc: 0.95853	valid_set's atec_metric: 0.437791
[15]	train_set's auc: 0.96897	train_set's atec_metric: 0.565059	valid_set's auc: 0.960022	valid_set's atec_metric: 0.445543
[16]	train_set's auc: 0.969656	train_set's atec_metric: 0.569009	valid_set's auc: 0.962188	valid_set's atec_metric: 0.44845
[17]	train_set's auc: 0.970964	train_set's atec_metric: 0.575083	valid_set's auc: 0.963127	valid_set's atec_metric: 0.448934
[18]	train_set's auc: 0.971781	train_set's atec_metric: 0.5811	valid_set's auc: 0.963667	valid_set's atec_metric: 0.454651
[19]	train_set's auc: 0.973912	train_set's atec_metric: 0.58606	valid_set's auc: 0.965504	valid_set's atec_metric: 0.458624
[20]	train_set's auc: 0.974386	train_set's atec_metric: 0.59156	valid_set's auc: 0.965614	valid_set's atec_metric: 0.459787
[21]	tra

[6]	train_set's auc: 0.960582	train_set's atec_metric: 0.513331	valid_set's auc: 0.949951	valid_set's atec_metric: 0.388566
[7]	train_set's auc: 0.960732	train_set's atec_metric: 0.521839	valid_set's auc: 0.949995	valid_set's atec_metric: 0.397771
[8]	train_set's auc: 0.961596	train_set's atec_metric: 0.525307	valid_set's auc: 0.951159	valid_set's atec_metric: 0.405911
[9]	train_set's auc: 0.963956	train_set's atec_metric: 0.532128	valid_set's auc: 0.953107	valid_set's atec_metric: 0.409496
[10]	train_set's auc: 0.964483	train_set's atec_metric: 0.54214	valid_set's auc: 0.954868	valid_set's atec_metric: 0.425291
[11]	train_set's auc: 0.965048	train_set's atec_metric: 0.550396	valid_set's auc: 0.957148	valid_set's atec_metric: 0.431395
[12]	train_set's auc: 0.965665	train_set's atec_metric: 0.557515	valid_set's auc: 0.957901	valid_set's atec_metric: 0.439922
[13]	train_set's auc: 0.968032	train_set's atec_metric: 0.562889	valid_set's auc: 0.958446	valid_set's atec_metric: 0.448934
[14]	

...

[62]	train_set's auc: 0.981467	train_set's atec_metric: 0.644643	valid_set's auc: 0.968321	valid_set's atec_metric: 0.491085
[63]	train_set's auc: 0.981632	train_set's atec_metric: 0.646573	valid_set's auc: 0.968289	valid_set's atec_metric: 0.493605
[64]	train_set's auc: 0.981719	train_set's atec_metric: 0.648065	valid_set's auc: 0.968322	valid_set's atec_metric: 0.495349
[65]	train_set's auc: 0.981895	train_set's atec_metric: 0.650878	valid_set's auc: 0.968011	valid_set's atec_metric: 0.497287
[66]	train_set's auc: 0.981976	train_set's atec_metric: 0.651808	valid_set's auc: 0.967218	valid_set's atec_metric: 0.497481
[67]	train_set's auc: 0.982637	train_set's atec_metric: 0.655081	valid_set's auc: 0.967801	valid_set's atec_metric: 0.496318
Early stopping, best iteration is:
[57]	train_set's auc: 0.98078	train_set's atec_metric: 0.634964	valid_set's auc: 0.968545	valid_set's atec_metric: 0.491957
[1]	train_set's auc: 0.936234	train_set's atec_metric: 0.42401	valid_set's auc: 0.89801	val

[60]	train_set's auc: 0.981129	train_set's atec_metric: 0.646607	valid_set's auc: 0.96796	valid_set's atec_metric: 0.483236
[61]	train_set's auc: 0.981429	train_set's atec_metric: 0.648525	valid_set's auc: 0.969093	valid_set's atec_metric: 0.485756
[62]	train_set's auc: 0.981548	train_set's atec_metric: 0.651544	valid_set's auc: 0.96927	valid_set's atec_metric: 0.485465
[63]	train_set's auc: 0.981685	train_set's atec_metric: 0.653255	valid_set's auc: 0.968892	valid_set's atec_metric: 0.486143
[64]	train_set's auc: 0.981777	train_set's atec_metric: 0.655311	valid_set's auc: 0.969061	valid_set's atec_metric: 0.488275
Early stopping, best iteration is:
[54]	train_set's auc: 0.980053	train_set's atec_metric: 0.634355	valid_set's auc: 0.968265	valid_set's atec_metric: 0.491085
[1]	train_set's auc: 0.934009	train_set's atec_metric: 0.387266	valid_set's auc: 0.8978	valid_set's atec_metric: 0.288372
Training until validation scores don't improve for 10 rounds.
[2]	train_set's auc: 0.933844	tra

[61]	train_set's auc: 0.960171	train_set's atec_metric: 0.482685	valid_set's auc: 0.949558	valid_set's atec_metric: 0.399128
[62]	train_set's auc: 0.960198	train_set's atec_metric: 0.483086	valid_set's auc: 0.94958	valid_set's atec_metric: 0.402035
[63]	train_set's auc: 0.960331	train_set's atec_metric: 0.483775	valid_set's auc: 0.949728	valid_set's atec_metric: 0.401938
Early stopping, best iteration is:
[53]	train_set's auc: 0.959262	train_set's atec_metric: 0.476852	valid_set's auc: 0.947827	valid_set's atec_metric: 0.403876
[1]	train_set's auc: 0.935184	train_set's atec_metric: 0.398599	valid_set's auc: 0.898257	valid_set's atec_metric: 0.290698
Training until validation scores don't improve for 10 rounds.
[2]	train_set's auc: 0.935207	train_set's atec_metric: 0.416374	valid_set's auc: 0.89828	valid_set's atec_metric: 0.295349
[3]	train_set's auc: 0.9355	train_set's atec_metric: 0.416604	valid_set's auc: 0.898386	valid_set's atec_metric: 0.297093
[4]	train_set's auc: 0.942671	train

[5]	train_set's auc: 0.943011	train_set's atec_metric: 0.453003	valid_set's auc: 0.904096	valid_set's atec_metric: 0.291182
[6]	train_set's auc: 0.943011	train_set's atec_metric: 0.452279	valid_set's auc: 0.90405	valid_set's atec_metric: 0.291182
[7]	train_set's auc: 0.943047	train_set's atec_metric: 0.455265	valid_set's auc: 0.904091	valid_set's atec_metric: 0.349709
[8]	train_set's auc: 0.943065	train_set's atec_metric: 0.451751	valid_set's auc: 0.904128	valid_set's atec_metric: 0.353682
[9]	train_set's auc: 0.943072	train_set's atec_metric: 0.454116	valid_set's auc: 0.904101	valid_set's atec_metric: 0.36657
[10]	train_set's auc: 0.943106	train_set's atec_metric: 0.456884	valid_set's auc: 0.904212	valid_set's atec_metric: 0.36657
[11]	train_set's auc: 0.944304	train_set's atec_metric: 0.454312	valid_set's auc: 0.90482	valid_set's atec_metric: 0.369671
[12]	train_set's auc: 0.945044	train_set's atec_metric: 0.458732	valid_set's auc: 0.911336	valid_set's atec_metric: 0.368895
[13]	trai

[17]	train_set's auc: 0.945929	train_set's atec_metric: 0.482914	valid_set's auc: 0.912527	valid_set's atec_metric: 0.376066
[18]	train_set's auc: 0.945938	train_set's atec_metric: 0.482019	valid_set's auc: 0.912548	valid_set's atec_metric: 0.375775
[19]	train_set's auc: 0.946005	train_set's atec_metric: 0.48195	valid_set's auc: 0.91319	valid_set's atec_metric: 0.375872
[20]	train_set's auc: 0.946008	train_set's atec_metric: 0.486749	valid_set's auc: 0.913209	valid_set's atec_metric: 0.371221
[21]	train_set's auc: 0.946049	train_set's atec_metric: 0.488001	valid_set's auc: 0.913231	valid_set's atec_metric: 0.372384
[22]	train_set's auc: 0.946113	train_set's atec_metric: 0.489884	valid_set's auc: 0.913357	valid_set's atec_metric: 0.371996
[23]	train_set's auc: 0.946137	train_set's atec_metric: 0.491848	valid_set's auc: 0.913366	valid_set's atec_metric: 0.368314
[24]	train_set's auc: 0.946154	train_set's atec_metric: 0.492307	valid_set's auc: 0.913385	valid_set's atec_metric: 0.367926
[2

In [41]:
print(best_score,best_params)

0.4996124031007752 {'boosting_type': 'gbdt', 'device': 'cpu', 'learning_rate': 0.05, 'metric': 'auc', 'num_leaves': 35, 'objective': 'binary', 'train_metric': True, 'verbose': 0}
